Note of things to fix: 
- Address extractor doesn't work if words do not have spaces between them

In [8]:
#testing uploading pdf
import PyPDF2
import io
import pyap

def extract_pdf_content(content: bytes) -> str:
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(content))
        # pdf_reader = PyPDF2.PdfReader(content)
        text = ""
        for i, page in enumerate(pdf_reader.pages):
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print('error with file')
        #adlflafj
        raise

Testing Reading & Removing Senstiv e info from pdf file

In [9]:

pdf_file = open("../statements/bank_statement_BofA.pdf", "rb")
pdf_content= extract_pdf_content(pdf_file.read())

#get bank
bank= input("What is your bank?")
#removing account number(s)
num_of_accounts= input("How many account numbers do you have?")
all_account_nums=[]
for i in num_of_accounts:
    if bank=="Bank of America":
        all_account_nums.append(input("What is you account number. Please provide a space between every 4 numbers."))
    else:
        all_account_nums.append(input("What is you account number."))

#remove all of the account numbers
print(all_account_nums)
for num in all_account_nums:
    pdf_content= pdf_content.replace(num, "")


#removing address from pdf content
all_address= str(pyap.parse(pdf_content, country='US'))[1:-1]
all_address= all_address.replace(",", "")
for address_icon in all_address.split(" "):
    pdf_content= pdf_content.replace(address_icon, "")

# #removing account number(s)
# num_of_accounts= input("How many account numbers do you have?")
# all_account_nums=[]
# for i in num_of_accounts:
#     all_account_nums.append(input("What is your account number?"))

# for num in all_account_nums:
#     all_address= all_address.replace(num, "")


print(pdf_content)

['1324 2323 1234']
Customer service information
1.888.888.RWDS (1.888.888.7937)
En Español: 1.800.688.6086
bankofamerica.com
Bank of America, N.A.
P.O. Box 25118
Tampa, FL 33622-5118YABETSE E GEBREWOLD
MARTHA TADESSE CHONCHORO
   
,   P.O. Box 15284
Wilmington, DE 19850
 PULL: B   CYCLE: 11   SPEC: E   DELIVERY: E   TYPE:    IMAGE: I   BC: 
 Page 1 of 4Your Bank of America Advantage Savings 
Preferred Rewards Gold
for June 15, 2024 to July 18, 2024 Account number: 4460 4914 2758
YABETSE E GEBREWOLD     MARTHA TADESSE CHONCHORO     
Account summary
Beginning balance on June 15, 2024 $21.00
Deposits and other additions 60.00
ATM and debit card subtractions -0.00
Other subtractions -0.00
Service fees -0.00
Ending balance on July 18, 2024 $81.00

YABETSE E GEBREWOLD   !   Account # 4460 4914 2758   !   June 15, 2024 to July 18, 2024
Page 2 of 4IMPORTANT INFORMATION:
BANK DEPOSIT ACCOUNTS
How to Contact Us - You may call us at the telephone number listed on the front of this statement.
Upda

Creating Read & Removing Info as a Function

In [10]:
#creating above read & redact code into a function
def read_redact_file(file_name, all_account_nums):
    pdf_content= extract_pdf_content(pdf_file.read())

    #remove all of the account numbers
    #print(all_account_nums)
    for num in all_account_nums:
        pdf_content= pdf_content.replace(num, "")

    #removing address from pdf content
    all_address= str(pyap.parse(pdf_content, country='US'))[1:-1]
    # if all_address == "":
    #     all_address = input("Type you address")
    all_address= all_address.replace(",", "")
    for address_icon in all_address.split(" "):
        pdf_content= pdf_content.replace(address_icon, "")
    return (all_address, pdf_content)

In [11]:
#code for reading all statements in statement folder
import os
folder = '../statements'
files = os.listdir(folder)

outputs = []

#get bank
bank= input("What is your bank?")

#removing account number(s)
num_of_accounts= int(input("How many account numbers do you have?"))
all_account_nums=[]

for i in range(num_of_accounts):
    if bank=="Bank of America":
        all_account_nums.append(input("What is you account number. Please provide a space between every 4 numbers."))
    else:
        all_account_nums.append(input("What is you account number."))

#goes through each file
for statement in files:
    pdf_file = open(f'../statements/{statement}', "rb")
    outputs.append(read_redact_file(pdf_file, all_account_nums))

In [12]:
outputs

[('18842 BROKEN OAK RD BOYDS MD 20841-4222',
  'Customer service information\n1.888.888.RWDS (1.888.888.7937)\nEn Español: 1.800.688.6086\nbankofamerica.com\nBank of America, N.A.\nP.O. Box 25118\nTampa, FL 33622-5118YABETSE E GEBREWOLD\nMARTHA TADESSE CHONCHORO\n   \n,   P.O. Box 15284\nWilmington, DE 19850\n PULL: B   CYCLE: 11   SPEC: E   DELIVERY: E   TYPE:    IMAGE: I   BC: \n Page 1 of 4Your Bank of America Advantage Savings \nPreferred Rewards Gold\nfor June 15, 2024 to July 18, 2024 Account number: 4460 4914 2758\nYABETSE E GEBREWOLD     MARTHA TADESSE CHONCHORO     \nAccount summary\nBeginning balance on June 15, 2024 $21.00\nDeposits and other additions 60.00\nATM and debit card subtractions -0.00\nOther subtractions -0.00\nService fees -0.00\nEnding balance on July 18, 2024 $81.00\n\nYABETSE E GEBREWOLD   !   Account # 4460 4914 2758   !   June 15, 2024 to July 18, 2024\nPage 2 of 4IMPORTANT INFORMATION:\nBANK DEPOSIT ACCOUNTS\nHow to Contact Us - You may call us at the tele

In [13]:
string = 'hello worldhelphigreenblue'
print(string.split())

['hello', 'worldhelphigreenblue']


In [14]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
#make one context
all_statements=""
for statement in outputs:
    all_statements+=statement[1]

template= "Use the following pieces of content to anser the question at the end. \n\n{context}\n\nQuestion: {question}\nHelpful Answer"
prompt=ChatPromptTemplate.from_template(template)
# prompt= ChatPromptTemplate.from_messages([
#     ("system", "You are a financial advisor helping people save money"),
#     ("human", "{input}")
# ])

llm= Ollama(model= "llama3")
chain= prompt | llm
text = chain.invoke({"question": "Come up with a good savings plan. Based off my recent transactions.", "context":all_statements})
print(text)

Based on your recent transactions, I notice that you've had consistent deposits into your account over the past month. Here's a potential savings plan for you:

1. **Regular Deposits**: Continue to set aside $30 every 2-3 days using online banking transfers from your other accounts (CHK 7782). This habit will help you build a stable foundation for your savings.
2. **Target Savings Goal**: Since you've been consistently depositing $60 per month, consider setting a target savings goal for the next quarter or semester. For example, aim to save an additional $100-200 by the end of September.
3. **Automate Your Savings**: To make saving easier and less prone to being neglected, set up automatic transfers from your checking account to your savings account. You can schedule these transfers on a specific date (e.g., every 2nd and 4th Friday) or based on your paycheck schedule.
4. **Monitor Progress**: Regularly review your account statements to track your progress and make adjustments as neede